In [240]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
from numpy import array, concatenate
import numpy as np

/opt/anaconda3/envs/hospitalization_prediction/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """


##### Reference:  Multistep Time Series Forecasting with LSTMs in Python
by Jason Brownlee on May 10, 2017 in Deep Learning for Time Series
https://machinelearningmastery.com/multi-step-time-series-forecasting-long-short-term-memory-networks-python/

In [65]:
# convert time series into supervised learning problem
def series_to_supervised(data, n_in=1, n_out=1, n_features = 1,
                         pred_id = 0, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    
    # Drop non-predicting variables
    drop_cols = list(set(range(-n_features*n_out,0)) - \
                set([(-(i+1)*n_features) for i in range(n_out)]))
    
    agg = agg.drop(agg.columns[drop_cols], axis=1)
    return agg

In [66]:
df = read_csv('procecssed_df.csv', header=0, parse_dates=['date'],
             index_col=0)
df

,location,location_name,hospitalizations,smoothed_hosp,cases,smoothed_cases,spc,sph
date,,,,,,,,
2020-08-02,54,West Virginia,15,13.571429,120,138.428571,1.667824,0.135794
2020-08-03,54,West Virginia,16,12.714286,120,139.857143,1.613695,0.130553
2020-08-04,54,West Virginia,17,12.857143,81,134.142857,1.594363,0.130621
2020-08-05,54,West Virginia,10,13.285714,103,126.428571,1.578601,0.129031
2020-08-06,54,West Virginia,15,12.571429,116,129.000000,1.553070,0.126456
...,...,...,...,...,...,...,...,...
2021-11-09,12,Florida,209,197.571429,0,1499.714286,2.640175,0.186948
2021-11-10,12,Florida,197,196.571429,0,1499.714286,2.699973,0.188298
2021-11-11,12,Florida,172,193.000000,0,1499.714286,2.558361,0.188551


In [67]:
state = 'Florida'
state_df = df[df['location_name']==state]
state_df = state_df.drop(['location','location_name', 
                          'hospitalizations', 'cases'], axis=1)


state_supervised = series_to_supervised(state_df, n_in=21, n_out=7,
                                        n_features=4, pred_id=0)

print(f'Total Number of Days of one state: {len(state_df)},\n\
        Converted dataframe length: {len(state_supervised)}')
state_supervised

Total Number of Days of one state: 469,
        Converted dataframe length: 442


,var1(t-21),var2(t-21),var3(t-21),var4(t-21),var1(t-20),var2(t-20),var3(t-20),var4(t-20),var1(t-19),var2(t-19),...,var2(t-1),var3(t-1),var4(t-1),var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6)
date,,,,,,,,,,,,,,,,,,,,,
2020-08-23,727.428571,9039.571429,1.573534,0.142987,720.000000,8448.142857,1.557854,0.137681,706.000000,7907.571429,...,3994.285714,1.231201,0.121209,494.857143,486.714286,477.571429,469.285714,452.428571,441.428571,429.857143
2020-08-24,720.000000,8448.142857,1.557854,0.137681,706.000000,7907.571429,1.550967,0.135497,696.000000,7330.857143,...,3879.285714,1.220833,0.121471,486.714286,477.571429,469.285714,452.428571,441.428571,429.857143,417.857143
2020-08-25,706.000000,7907.571429,1.550967,0.135497,696.000000,7330.857143,1.521319,0.132901,679.857143,7001.428571,...,3819.285714,1.225841,0.121267,477.571429,469.285714,452.428571,441.428571,429.857143,417.857143,407.714286
2020-08-26,696.000000,7330.857143,1.521319,0.132901,679.857143,7001.428571,1.478156,0.127912,660.571429,6812.714286,...,3652.857143,1.230840,0.121599,469.285714,452.428571,441.428571,429.857143,417.857143,407.714286,397.000000
2020-08-27,679.857143,7001.428571,1.478156,0.127912,660.571429,6812.714286,1.449846,0.123292,651.142857,6649.857143,...,3525.000000,1.259533,0.120862,452.428571,441.428571,429.857143,417.857143,407.714286,397.000000,389.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-03,391.857143,3597.714286,3.078014,0.228869,376.571429,3597.714286,3.015098,0.227363,365.285714,2686.714286,...,1735.857143,2.561569,0.176048,208.714286,204.285714,203.714286,203.142857,204.142857,201.000000,197.571429
2021-11-04,376.571429,3597.714286,3.015098,0.227363,365.285714,2686.714286,2.942990,0.223296,352.000000,2686.714286,...,1735.857143,2.508921,0.175826,204.285714,203.714286,203.142857,204.142857,201.000000,197.571429,196.571429
2021-11-05,365.285714,2686.714286,2.942990,0.223296,352.000000,2686.714286,2.938174,0.221030,343.000000,2686.714286,...,1735.857143,2.523791,0.176612,203.714286,203.142857,204.142857,201.000000,197.571429,196.571429,193.000000


In [82]:
# transform series into train and test sets for supervised learning
def prepare_data(series, n_test, n_lag, n_seq, n_features, pred_id=0):
    # extract raw values
    raw_values = series.values
    raw_values = raw_values.reshape(len(raw_values), -1)
    
    # rescale values to -1, 1
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_values = scaler.fit_transform(raw_values)
    scaled_values = scaled_values.reshape(len(scaled_values), -1)
    # transform into supervised learning problem X, y
    supervised = series_to_supervised(scaled_values, n_lag, n_seq,
                                      n_features=n_features, 
                                      pred_id=pred_id)
    supervised_values = supervised.values
    # split into train and test sets
    train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
    return scaler, train, test

In [136]:
# Configure
n_features = 4
n_lag = 21
n_seq = 7
n_test = len(state_supervised) - 360 
# test = 88 days
# train = 354 days

# Prepare data
scaler, train, test = prepare_data(state_df, n_test=n_test,
                          n_lag=n_lag, n_seq=n_seq,
                          n_features=n_features, pred_id=0)

# n_feature * n_lag + n_seq
# 4         * 21    + 7     = 91
print('Train: %s, Test: %s' % (train.shape, test.shape))

Train: (360, 91), Test: (82, 91)


In [191]:
# fit an LSTM network to training data
def fit_lstm(train, n_lag, n_features, 
             n_seq, n_batch, nb_epoch, 
             n_neurons):
    # reshape training into [samples, timesteps, features]
    X, y = train[:, 0:n_lag*n_features], train[:, n_lag*n_features:]
    X = X.reshape(X.shape[0], n_lag, n_features)
    # design network
    model = Sequential()
    model.add(LSTM(n_neurons, activation='tanh',
                   return_sequences=True,
                   input_shape=(X.shape[1], X.shape[2])))
    model.add(LSTM(n_neurons, activation='tanh'))
    model.add(Dense(y.shape[1]))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    history = model.fit(X, y, epochs=nb_epoch, 
                        batch_size=n_batch,  
                        verbose=2, shuffle=False)
    
    return model

In [ ]:
# # fit an LSTM network to training data
# def fit_lstm(train, n_lag, n_features, 
#              n_seq, n_batch, nb_epoch, 
#              n_neurons):
#     # reshape training into [samples, timesteps, features]
#     X, y = train[:, 0:n_lag*n_features], train[:, n_lag*n_features:]
#     X = X.reshape(X.shape[0], n_lag, n_features)
#     # design network
#     model = Sequential()
#     model.add(LSTM(n_neurons, activation='tanh', return_sequences=True,
#                    batch_input_shape=(n_batch, X.shape[1], X.shape[2]), 
#                    stateful=True))
#     model.add(LSTM(n_neurons, activation='tanh'))
#     model.add(Dense(y.shape[1]))
#     model.compile(loss='mae', optimizer='adam')
#     # fit network
#     history = model.fit(X, y, epochs=nb_epoch, 
#                         batch_size=n_batch, verbose=2, shuffle=False)
    
#     return model

In [129]:
X, y = train[:, 0:n_lag*n_features], train[:, n_lag*n_features:]
X = X.reshape(X.shape[0], 1, X.shape[1])
y.shape

(360, 7)

In [192]:
model = fit_lstm(train, n_lag=n_lag, n_features=n_features,
                 n_seq=n_seq, n_batch=72, nb_epoch=200, n_neurons=64)

Epoch 1/200
5/5 - 2s - loss: 0.6369 - 2s/epoch - 346ms/step
Epoch 2/200
5/5 - 0s - loss: 0.4334 - 58ms/epoch - 12ms/step
Epoch 3/200
5/5 - 0s - loss: 0.2796 - 58ms/epoch - 12ms/step
Epoch 4/200
5/5 - 0s - loss: 0.1743 - 56ms/epoch - 11ms/step
Epoch 5/200
5/5 - 0s - loss: 0.1854 - 60ms/epoch - 12ms/step
Epoch 6/200
5/5 - 0s - loss: 0.1558 - 61ms/epoch - 12ms/step
Epoch 7/200
5/5 - 0s - loss: 0.1499 - 67ms/epoch - 13ms/step
Epoch 8/200
5/5 - 0s - loss: 0.1459 - 63ms/epoch - 13ms/step
Epoch 9/200
5/5 - 0s - loss: 0.1316 - 61ms/epoch - 12ms/step
Epoch 10/200
5/5 - 0s - loss: 0.1298 - 62ms/epoch - 12ms/step
Epoch 11/200
5/5 - 0s - loss: 0.1250 - 56ms/epoch - 11ms/step
Epoch 12/200
5/5 - 0s - loss: 0.1227 - 63ms/epoch - 13ms/step
Epoch 13/200
5/5 - 0s - loss: 0.1198 - 64ms/epoch - 13ms/step
Epoch 14/200
5/5 - 0s - loss: 0.1170 - 64ms/epoch - 13ms/step
Epoch 15/200
5/5 - 0s - loss: 0.1137 - 58ms/epoch - 12ms/step
Epoch 16/200
5/5 - 0s - loss: 0.1094 - 60ms/epoch - 12ms/step
Epoch 17/200
5/5 -

Epoch 133/200
5/5 - 0s - loss: 0.0423 - 58ms/epoch - 12ms/step
Epoch 134/200
5/5 - 0s - loss: 0.0428 - 60ms/epoch - 12ms/step
Epoch 135/200
5/5 - 0s - loss: 0.0350 - 58ms/epoch - 12ms/step
Epoch 136/200
5/5 - 0s - loss: 0.0331 - 60ms/epoch - 12ms/step
Epoch 137/200
5/5 - 0s - loss: 0.0353 - 60ms/epoch - 12ms/step
Epoch 138/200
5/5 - 0s - loss: 0.0428 - 58ms/epoch - 12ms/step
Epoch 139/200
5/5 - 0s - loss: 0.0354 - 61ms/epoch - 12ms/step
Epoch 140/200
5/5 - 0s - loss: 0.0373 - 61ms/epoch - 12ms/step
Epoch 141/200
5/5 - 0s - loss: 0.0325 - 60ms/epoch - 12ms/step
Epoch 142/200
5/5 - 0s - loss: 0.0342 - 59ms/epoch - 12ms/step
Epoch 143/200
5/5 - 0s - loss: 0.0378 - 58ms/epoch - 12ms/step
Epoch 144/200
5/5 - 0s - loss: 0.0459 - 59ms/epoch - 12ms/step
Epoch 145/200
5/5 - 0s - loss: 0.0381 - 59ms/epoch - 12ms/step
Epoch 146/200
5/5 - 0s - loss: 0.0378 - 59ms/epoch - 12ms/step
Epoch 147/200
5/5 - 0s - loss: 0.0317 - 60ms/epoch - 12ms/step
Epoch 148/200
5/5 - 0s - loss: 0.0307 - 60ms/epoch - 12

In [147]:
# evaluate the persistence model
def make_forecasts(model, n_batch, test, 
                   n_features, n_lag, n_seq):
    test_X, test_y = test[:, 0:n_lag*n_features], test[:, n_lag*n_features:]
    test_X = test_X.reshape((test_X.shape[0], n_lag, n_features))

    forecasts = model.predict(test_X, batch_size=n_batch)
    
    return forecasts

In [263]:
forecasts = make_forecasts(model, n_batch=72, test=test, 
                            n_features=n_features,
                            n_lag=n_lag, n_seq=n_seq)

forecasts.shape

(82, 7)

In [299]:
# inverse data transform on forecasts
def inverse_transform(test, forecasts, scaler, 
                      n_lag, n_features):
    test_X, test_y = test[:, 0:n_lag*n_features], test[:, n_lag*n_features:]    
    inverted_hat = np.zeros(shape=forecasets.shape)
    inverted_truth = np.zeros(shape=test_y.shape)
    for i in range(forecasets.shape[1]):
        # Inverse predictions
        y_hat_i = forecasets[:,[i]]
        inv_yhat_i = concatenate((y_hat_i, test_X[:, -(n_features-1):]), axis=1)
        inv_yhat_i = scaler.inverse_transform(inv_yhat_i)
        inv_yhat_i = inv_yhat_i[:,[0]]
        
        # store
        inverted_hat[:,[i]] = inv_yhat_i
        
        # Inverse Ground Truth
        y_i = test_y[:,[i]]
        inv_y_i = concatenate((y_i, test_X[:, -(n_features-1):]), axis=1)
        inv_y_i = scaler.inverse_transform(inv_y_i)
        inv_y_i = inv_y_i[:,[0]]
        
        # store
        inverted_truth[:,[i]] = inv_y_i
    return inverted_hat, inverted_truth

In [300]:
forecasets.shape

(82, 7)

In [304]:
# inverse transform forecasts and test
inv_forecasts, inv_truth = inverse_transform(test=test, forecasts=forecasts, 
                                                  scaler=scaler,n_lag=n_lag, 
                                                  n_features=n_features)
print(inv_forecasts.shape, inv_truth.shape)

(82, 7) (82, 7)


In [294]:
def evaluate_forecasts(test, forecasts, n_lag, n_seq):
    for i in range(n_seq):
        actual = [row[i] for row in test]
        predicted = [forecast[i] for forecast in forecasts]
        rmse = sqrt(mean_squared_error(actual, predicted))
        print('t+%d RMSE: %f' % ((i+1), rmse))

In [306]:
evaluate_forecasts(test=inv_truth, forecasts=inv_forecasts, 
                   n_lag=n_lag, n_seq=n_seq)

t+1 RMSE: 290.070510
t+2 RMSE: 305.293347
t+3 RMSE: 322.880500
t+4 RMSE: 366.696934
t+5 RMSE: 405.017323
t+6 RMSE: 405.170394
t+7 RMSE: 445.688753
